In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/1.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,akjmXXq,0,FEM,28.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,MV6b6R9,0,FEM,30.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,zvxv1Rm,0,FEM,22.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,8oqJzL,0,MASC,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ez533B,0,MASC,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

12377268
12332167


In [9]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
7472401,1111644641,EzEYQZ6,0,MASC,25.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
3789478,1112400256,ZDar1J8,1,FEM,24.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,True,False,True,NaN
308495,1112320479,wVk4Rdp,1,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
4482350,1112457303,EzZ9EZ0,1,FEM,40.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
9909889,1112448621,a1qAv9,0,MASC,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
3131447,1112389353,PmaEB3b,1,MASC,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
4178662,1112437305,EzEdwm8,1,FEM,37.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,2.0
9486946,1112341160,ZDreQqZ,0,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
3209577,1112383590,ZD2081P,1,MASC,25.0,-,-,-,-,Graduado,...,False,False,True,False,True,False,False,False,False,NaN
504604,1112439298,A3XARX5,1,NO_DECLARA,20.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,1.0


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
1668017,1111909577,RzrXazx,1,MASC,26.0,-,-,-,-,-,...,False,True,True,False,False,False,False,False,False,0.0
5722962,1112393416,5Q8Mkz,1,FEM,39.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,False,0.0
11420253,1112293191,8MaKqZW,0,FEM,29.0,-,-,-,Graduado,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
1335110,1112406694,3NdZe1a,1,MASC,23.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
8639192,1112406250,xkvrde6,0,MASC,43.0,-,-,-,-,Graduado,...,False,True,True,False,False,True,False,False,False,0.0
8841942,1112305562,eYKl22,0,FEM,35.0,-,-,-,-,Graduado,...,False,True,True,False,False,True,False,True,False,0.0
1685504,1112413622,YVeE4X,1,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,0.0
12258509,1112252496,aPlEdq,0,FEM,33.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
4903112,1112448068,QNMaMAN,1,FEM,35.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,1.0
6000998,1112442580,wVkJP4m,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,6.0


In [11]:
zonas=avisos['nombre_zona'].value_counts().index
postulaciones['nombre_zona']=postulaciones['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
postulaciones['tipo_de_trabajo']=postulaciones['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
postulaciones['nivel_laboral']=postulaciones['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
postulaciones['nombre_area']=postulaciones['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
postulaciones['Doctorado']=postulaciones['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
postulaciones['Master']=postulaciones['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
postulaciones['Otro']=postulaciones['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
postulaciones['Posgrado']=postulaciones['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
postulaciones['Secundario']=postulaciones['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
postulaciones['Terciario/Técnico']=postulaciones['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
postulaciones['Universitario']=postulaciones['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
postulaciones['sexo']=postulaciones['sexo'].replace(sexo, range(len(sexo)))
postulaciones.dtypes

idaviso                        int64
idpostulante                  object
sepostulo                      int64
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [12]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

Train:  11098950 Test:  1233217


In [14]:
c=XGBClassifier(objective='binary:logistic',max_depth=13,learning_rate=0.05,subsample=0.8,colsample_bytree=0.8,n_estimators=500,silent=False,n_jobs=-1,verbose=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl"
dt=c.fit(x_train,y_train)

In [15]:
score=c.score(x_test,y_test)*100
print(score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


74.59960412482151


In [16]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/XG2.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/XG2.pkl']

In [17]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
37704,71541,1112462824,ZjMrQE,FEM,38.0,-,-,-,-,-,...,False,True,False,False,False,True,False,False,False,0.0
59066,7790,1111736801,129aAj,MASC,38.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
93186,92340,1112474819,5oaezz,MASC,30.0,-,En Curso,-,-,Graduado,...,False,False,False,False,True,False,False,False,False,0.0
53518,75608,1112464159,bOjBLQV,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,0.0
61731,60533,1112455044,5q439q,FEM,39.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
62997,10562,1111921042,eQPlwo,MASC,41.0,-,-,Graduado,Graduado,Graduado,...,False,True,False,False,False,False,True,False,True,0.0
63578,10842,1111930198,awxD1N,FEM,36.0,-,-,-,-,-,...,False,False,True,False,True,False,False,False,False,0.0
48823,89874,1112467076,DrpzXRA,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
12131,27361,1112349978,vVDBd9Q,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,3.0
61181,9142,1111818415,YYwRdJ,MASC,35.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,False,0.0


In [18]:
zonas=avisos['nombre_zona'].value_counts().index
prediccion['nombre_zona']=prediccion['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
prediccion['tipo_de_trabajo']=prediccion['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
prediccion['nivel_laboral']=prediccion['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
prediccion['nombre_area']=prediccion['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
prediccion['Doctorado']=prediccion['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
prediccion['Master']=prediccion['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
prediccion['Otro']=prediccion['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
prediccion['Posgrado']=prediccion['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
prediccion['Secundario']=prediccion['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
prediccion['Terciario/Técnico']=prediccion['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
prediccion['Universitario']=prediccion['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
prediccion['sexo']=prediccion['sexo'].replace(sexo, range(len(sexo)))
prediccion.dtypes

id                             int64
idaviso                        int64
idpostulante                  object
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [19]:
x_final=prediccion[features]
y_final=c.predict(x_final)
y_final

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [20]:
prediccion['sepostulo']=y_final

In [21]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,0,42.0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0
1,1,739260,6v1xdL,1,30.0,0,0,1,0,1,...,False,False,False,False,False,False,False,False,0.0,0
2,2,739260,ezRKm9,0,36.0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0
3,3,758580,1Q35ej,1,68.0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,0.0,0
4,4,758580,EAN4J6,0,32.0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,0.0,0


In [22]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [23]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion10.csv",index=False)

In [24]:
prediccion.sepostulo.mean()

0.53762

In [25]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
31307,56457,1112448955,Z6LakV,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
4599,78234,1112464507,eMr48j,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
75136,54642,1112446858,Z6xQaR,MASC,49.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
27798,67454,1112460307,EOwaa0,FEM,44.0,-,-,-,-,Graduado,...,False,True,False,False,True,True,False,False,False,1.0
50525,63069,1112457238,NzrvAlJ,NO_DECLARA,200.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,1.0
37757,73653,1112463420,eJO4zr,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
31262,51794,1112442092,ow2MYGZ,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,True,True,True,False,True,1.0
17995,73316,1112463356,rm2L5BA,MASC,22.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
31630,24643,1112320916,ZDrp2PY,MASC,40.0,-,-,-,-,Abandonado,...,False,False,False,True,False,False,True,False,True,4.0
40812,46471,1112430035,bOzXQ29,MASC,26.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0


In [26]:
zonas=avisos['nombre_zona'].value_counts().index
prediccion['nombre_zona']=prediccion['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
prediccion['tipo_de_trabajo']=prediccion['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
prediccion['nivel_laboral']=prediccion['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
prediccion['nombre_area']=prediccion['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
prediccion['Doctorado']=prediccion['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
prediccion['Master']=prediccion['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
prediccion['Otro']=prediccion['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
prediccion['Posgrado']=prediccion['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
prediccion['Secundario']=prediccion['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
prediccion['Terciario/Técnico']=prediccion['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
prediccion['Universitario']=prediccion['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
prediccion['sexo']=prediccion['sexo'].replace(sexo, range(len(sexo)))
prediccion.dtypes

id                             int64
idaviso                        int64
idpostulante                  object
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [27]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.827799  , 0.172201  ],
       [0.6335703 , 0.3664297 ],
       [0.63655794, 0.36344206],
       ...,
       [0.8823969 , 0.11760313],
       [0.68781817, 0.3121818 ],
       [0.96371955, 0.03628047]], dtype=float32)

In [28]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [29]:
prediccion['sepostulo']=y_final2

In [30]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,0,42.0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0.172201
1,1,739260,6v1xdL,1,30.0,0,0,1,0,1,...,False,False,False,False,False,False,False,False,0.0,0.366430
2,2,739260,ezRKm9,0,36.0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0.363442
3,3,758580,1Q35ej,1,68.0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,0.0,0.087805
4,4,758580,EAN4J6,0,32.0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,0.0,0.307014


In [31]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.172201
1,1,0.366430
2,2,0.363442
3,3,0.087805
4,4,0.307014


In [32]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion11.csv",index=False)

In [33]:
prediccion.sepostulo.mean()

0.5965679032156757